In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 5.2 MB/s 
     |████████████████████████████████| 384 kB 39.9 MB/s 
     |████████████████████████████████| 140 kB 42.6 MB/s 
     |████████████████████████████████| 58 kB 4.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http:/

In [ ]:
# -*- coding: utf-8 -*-
"""Download_images.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1OcELHfJsM1YrIvMg31UTMECJw8DDd3i3
"""

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import json
import os
import errno
from multiprocessing import Pool
from pathlib import Path
import ssl
import time
import urllib
from urllib.request import urlopen
from tqdm import tqdm
import shutil
from pathlib import Path
import requests


dataset_path = '/content/drive/MyDrive/MedPix Dataset'



links_dict = {
             'AP': "https://medpix.nlm.nih.gov/search?pln=%5B%22AP%22%5D&allen=true&allt=true&alli=true",
            'Axical' : "https://medpix.nlm.nih.gov/search?pln=%5B%22Axial%22%5D&allen=true&allt=true&alli=true",
            'Coronal' : "https://medpix.nlm.nih.gov/search?pln=%5B%22Coronal%22%5D&allen=true&allt=true&alli=true",
             'Frontal' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Frontal%22%5D&allen=true&allt=true&alli=true',
              'Histology' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Histology%22%5D&allen=true&allt=true&alli=true',
              'Image-Plane' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Image%20Plane%22%5D&allen=true&allt=true&alli=true',
              'Lateral' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Lateral%22%5D&allen=true&allt=true&alli=true',
              'Longitudinal' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Longitudinal%22%5D&allen=true&allt=true&alli=true',
              'Mammo-CC' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Mammo%20-%20CC%22%5D&allen=true&allt=true&alli=true',
              'Mammo-MagCC' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Mammo%20-%20Mag%20CC%22%5D&allen=true&allt=true&alli=true',
              'Mammo-MLO' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Mammo%20-%20MLO%22%5D&allen=true&allt=true&alli=true',
              'Multiple-or-Montage' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Multiple%20or%20Montage%22%5D&allen=true&allt=true&alli=true',
              'NOS' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22NOS%20-%20Not%20specified%22%5D&allen=true&allt=true&alli=true',
              'Oblique' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Oblique%22%5D&allen=true&allt=true&alli=true',
              'Other-view' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Other%20View%20(see%20caption)%22%5D&allen=true&allt=true&alli=true',
              'PA' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22PA%22%5D&allen=true&allt=true&alli=true',
              'Sagittal' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Sagittal%22%5D&allen=true&allt=true&alli=true',
              'Transverse' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Transverse%22%5D&allen=true&allt=true&alli=true',
              '3D-Reconstruction' : 'https://medpix.nlm.nih.gov/search?pln=%5B%223D%20Reconstruction%22%5D&allen=true&allt=true&alli=true'
}

def save_to_json_file(current_path,new_data):
  myfile = Path(current_path +'/new_dataset.json')
  myfile.touch(exist_ok=True)
  with open(f'{current_path}/new_dataset.json', 'r', encoding='utf-8') as f:
    try:
        data = json.load(f)
    except ValueError:
         data = []
  with open(f'{current_path}/new_dataset.json', 'w', encoding='utf-8') as f:
    data = data + new_data
    json.dump(data, f, ensure_ascii=False, indent=4)

path_to_file = dataset_path+'/checkpoint.txt'
path = Path(path_to_file)
print(path.is_file())

if path.is_file():
    with open(dataset_path+'/checkpoint.txt','r') as f:
        lines = f.readlines()
        curr_plane , index = lines[0].strip() , int(lines[1])
        print(curr_plane, index)
else:
    curr_plane , index = None , 0

for plane in links_dict:
    print(curr_plane)
    if (not curr_plane is None) and (plane != curr_plane):
        continue
    print('hereee')
    link = links_dict[plane]
    current_path = dataset_path+'/'+plane
    download_path = current_path + "/new_images"
    os.makedirs(download_path, exist_ok=True)
    with open(current_path+'/image_links.txt', 'r', encoding='utf-8') as f:
        links = f.readlines()
        links = list(set(links))
        links = links[index:]
        data = []
        driver = webdriver.Chrome('chromedriver',options=chrome_options)
        wait = WebDriverWait(driver, 15)
        print('plane: '+plane+' , '+str(len(links)))
        for link in links:
            if link is None:
                continue
            print('link '+str(index+1)+' ....')
            try:
                driver.get(link)
            except:
                try:
                    driver.get(link)
                except:
                    pass
            finally:
                with open(dataset_path+'/checkpoint.txt','w') as f:
                    f.write(plane + "\n ")
                    f.write(str(index)+"\n")
            time.sleep(5)  # Allow 4 seconds for the web page to open

            title = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'image-title')))
            title = title.text
            image_link = driver.find_element(by=By.CLASS_NAME, value='image-frame').find_element(by=By.TAG_NAME, value='img')
            image_link = image_link.get_attribute('src')

            contents = driver.find_element(by=By.ID, value="left-section")
            contents = contents.text.split('\n')
            caption = contents[contents.index('Caption') + 1] if "Caption" in contents else None
            plane = contents[contents.index('Plane') + 1] if "Plane" in contents else None
            modality = contents[contents.index('Modality') + 1] if "Modality" in contents else None
            demographics = contents[contents.index('Demographics') + 1] if "Demographics" in contents else None

            topic_sec = driver.find_element(by=By.LINK_TEXT, value="Topic")
            if topic_sec is None:
                continue
            topic_sec.click()
            time.sleep(3)
            contents = driver.find_element(by=By.ID, value="image-tab-content")
            if contents.text != 'No related topic found.':
                contents = driver.find_element(by=By.ID, value="image-tab-content").text
                contents = contents.split('\n')
                topic = contents[contents.index('TOPIC') + 1] if "TOPIC" in contents else None
                disease_discussion = contents[contents.index('Disease Discussion') + 1] if "Disease Discussion" in contents else None
                location = contents[contents.index('Location') + 1] if "Location" in contents else None
                category = contents[contents.index('Category') + 1] if "Category" in contents else None
                refrence_idx = contents.index('Reference') if 'Reference' in contents else len(contents)
                keywords = []
                if "Keywords" in contents:
                    if "Category" in contents:
                      keywords = contents[contents.index('Category') + 1 : refrence_idx ]
            else:
                topic = None; disease_discussion = None; location = None; category = None ; keywords = []

            entry = { 'id': index, 'title' : title, 'image_link' : image_link, 'caption': caption, 'plane' : plane, 'modality' : modality, 'demographics': demographics
                 ,'topic': topic, 'disease_discussion' : disease_discussion, 'location': location, 'category': category}


            if image_link is None:
                continue

            save_to_json_file(current_path, [entry])


            img_data = requests.get(image_link).content
            with open(download_path+'/'+plane+'_'+str(index)+'.jpg', 'wb') as handler:
                handler.write(img_data)

            index += 1

        curr_plane , index = None , 0

False
None
hereee


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


plane: Decubitus , 17
link 1 ....
link 2 ....
link 3 ....
link 4 ....
link 5 ....
link 6 ....
link 7 ....
link 8 ....
link 9 ....
link 10 ....
link 11 ....
link 12 ....
link 13 ....
link 14 ....
link 15 ....
link 16 ....
link 17 ....


In [ ]:
directory_list = ['AP', 'Axial', 'Coronal', 'Frontal', 'Histology', 'Image-Plane','Lateral','Longitudinal',
                  'Mammo-CC','Mammo-MagCC','Mammo-MLO','Multiple or Montage','NOS','Oblique','Other view','PA','3D Reconstruction', 'Transverse', 'Sagittal', 'Decubitus' ]
sum = 0
for plane in directory_list:
  current_path = dataset_path+'/'+plane
  with open(f'{current_path}/new_dataset.json', 'r', encoding='utf-8') as f:
    try:
        data = json.load(f)
    except ValueError:
        data = []
  sum += len(data)
  print(f'{plane} : {len(data)}')
print(sum)

In [ ]:
directory_list = ['AP', 'Axial', 'Coronal', 'Frontal', 'Histology', 'Image-Plane','Lateral','Longitudinal',
                  'Mammo-CC','Mammo-MagCC','Mammo-MLO','Multiple or Montage','NOS','Oblique','Other view','PA','3D Reconstruction', 'Transverse', 'Sagittal', 'Decubitus' ]
sum = 0
for plane in directory_list:
  current_path = dataset_path+'/'+plane + '/' +
image_list = list(glob.glob(path + 'train/*.jpg'))
len(image_list)

In [ ]:
import re
import json
import os
import errno
from multiprocessing import Pool
from pathlib import Path
import ssl
import time
import urllib
from urllib.request import urlopen
from tqdm import tqdm
import shutil
from pathlib import Path
import requests
import glob


dataset_path = '/content/drive/MyDrive/MedPix Dataset'

In [ ]:
def save_to_json_file(current_path,new_data):
  myfile = Path(current_path +'/r_dataset.json')
  myfile.touch(exist_ok=True)
  with open(f'{current_path}/r_dataset.json', 'r', encoding='utf-8') as f:
    try:
        data = json.load(f)
    except ValueError:
         data = []
  with open(f'{current_path}/r_dataset.json', 'w', encoding='utf-8') as f:
    data = data + new_data
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
directory_list = ['AP', 'Axial', 'Coronal', 'Frontal', 'Histology', 'Image-Plane','Lateral','Longitudinal',
                  'Mammo-CC','Mammo-MagCC','Mammo-MLO','Multiple or Montage','NOS','Oblique','Other view','PA','3D Reconstruction', 'Transverse', 'Sagittal', 'Decubitus' ]

for plane in directory_list:
  print(plane)
  current_path = dataset_path+'/'+plane
  image_list = list(glob.glob(current_path + '/new_images/*.jpg'))
  with open(f'{current_path}/r_dataset.json', 'r', encoding='utf-8') as f:
    try:
        data = json.load(f)
    except ValueError:
        data = []
  print(len(data))
  print(len(image_list))

AP
1446
1446
Axial
7250
7250
Coronal
2824
2824
Frontal
1016
1016
Histology
348
348
Image-Plane
182
182
Lateral
1165
1165
Longitudinal
300
300
Mammo-CC
116
116
Mammo-MagCC
12
12
Mammo-MLO
113
113
Multiple or Montage
320
320
NOS
926
926
Oblique
382
382
Other view
423
423
PA
656
656
3D Reconstruction
232
232
Transverse
488
488
Sagittal
3031
3031
Decubitus
17
17


In [ ]:
directory_list = ['AP', 'Axial', 'Coronal', 'Frontal', 'Histology', 'Image-Plane','Lateral','Longitudinal',
                  'Mammo-CC','Mammo-MagCC','Mammo-MLO','Multiple or Montage','NOS','Oblique','Other view','PA','3D Reconstruction', 'Transverse', 'Sagittal', 'Decubitus' ]

for plane in directory_list:
  print(plane)
  current_path = dataset_path+'/'+plane
  with open(f'{current_path}/new_dataset.json', 'r', encoding='utf-8') as f:
    try:
        data = json.load(f)
    except ValueError:
        data = []
  print(len(data))
  image_list = list(glob.glob(current_path + '/new_images/*.jpg'))
  image_ids = []
  for image in image_list:
    image_ids.append(int(image.split('/')[-1].split('.')[0].split('_')[-1]))

  new_data = []
  for i in range(len(data)):
    if data[i]['id'] in image_ids:
      new_data.append(data[i])
  print(len(new_data))
  save_to_json_file(current_path,new_data)




AP
1446
1446
Axial
7251
7251
Coronal
5649
5649
Frontal
1016
1016
Histology
349
349
Image-Plane
182
182
Lateral
1165
1165
Longitudinal
303
303
Mammo-CC
116
116
Mammo-MagCC
12
12
Mammo-MLO
113
113
Multiple or Montage
320
320
NOS
927
927
Oblique
767
767
Other view
846
846
PA
1313
1313
3D Reconstruction
232
232
Transverse
490
490
Sagittal
4357
4357
Decubitus
17
17


In [ ]:
directory_list = ['AP', 'Axial', 'Coronal', 'Frontal', 'Histology', 'Image-Plane','Lateral','Longitudinal',
                  'Mammo-CC','Mammo-MagCC','Mammo-MLO','Multiple or Montage','NOS','Oblique','Other view','PA','3D Reconstruction', 'Transverse', 'Sagittal', 'Decubitus' ]

for plane in directory_list:
  print(plane)
  current_path = dataset_path+'/'+plane
  with open(f'{current_path}/cleaned_dataset.json', 'r', encoding='utf-8') as f:
    try:
        data = json.load(f)
    except ValueError:
        data = []
  print(len(data))

  added = []
  new_data = []
  for i in range(len(data)):
    if data[i]['id'] not in added:
      new_data.append(data[i])
      added.append(data[i]['id'])
  print(len(new_data))
  save_to_json_file(current_path,new_data)




AP
1446
1446
Axial
7251
7250
Coronal
5649
2824
Frontal
1016
1016
Histology
349
348
Image-Plane
182
182
Lateral
1165
1165
Longitudinal
303
300
Mammo-CC
116
116
Mammo-MagCC
12
12
Mammo-MLO
113
113
Multiple or Montage
320
320
NOS
927
926
Oblique
767
382
Other view
846
423
PA
1313
656
3D Reconstruction
232
232
Transverse
490
488
Sagittal
4357
3031
Decubitus
17
17
